In [ ]:
#importar librerias
import torch
import torch.nn as nn
import torch.optim as optim #stochastic, adam, etc
import torch.nn.functional as F # funciones de activacion como relu o softmax
from torch.utils.data import DataLoader #manejo de dataset, creacion de minibatches
import torchvision.datasets as datasets #datasets de ejemplos
import torchvision.transforms as transforms #transformaciones que podemos hacer en el dataset

In [ ]:
#crear red fully-connected
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

#model = NN(784,10) # 28*28, 10 clases
#x = torch.randn(64, 784)
#print(model(x).shape)

In [ ]:
#configurar dispositivo donde se ejecutara
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#hiperparametros
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [ ]:
#cargando datos
train_dataset = datasets.MNIST(root="dataset/",train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root="dataset/",train=False, transform=transforms.ToTensor(), download=True)
test_loader  = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [ ]:
#inicializar la red
model = NN(input_size=input_size, num_classes = num_classes).to(device)

In [ ]:
#funcion de perdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#entrenar la red
for epoch in range(num_epochs): #cuantas veces la red ve la imagen
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    #print(data.shape) #torch.Size([64, 1, 28, 28]) 64 es el numero de batch, 1 es el canal de iamgen, 28 x 28 es la imagen entonces hay que aplanarlo
    data = data.reshape(data.shape[0], -1)
    #print(data.shape) #torch.Size([64, 784])


    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad() # configuramos el gradiente a 0 para cada paso de batch, asi no queda registrado el anterior. ojo que es el batch
    loss.backward()

    #descenso del gradiente o adam step
    optimizer.step()





In [ ]:
#chequear accuracy en el entrenamiento y en la prueba para ver que tan bueno es el modelo
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Chequeando en training set")
  else:
    print("chequeando en test set")
  num_correct =  0
  num_samples = 0
  model.eval()

  acc = 0
  with torch.no_grad(): #los calculos estan listos, asi que no hay que calcularlos
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1) #max de entre las 10 clases
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"obtuve {num_correct}/{num_samples} con accuracy {float(num_correct)/ float(num_samples)*100:.2f}")
    acc = float(num_correct)/ float(num_samples)*100
  model.train()
  return acc


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Chequeando en training set
obtuve 55850/60000 con accuracy 93.08
chequeando en test set
obtuve 9317/10000 con accuracy 93.17


93.17